In [22]:
import folium
import os
import sys

currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from hashed_similarities.grid_similarity import (
    generate_grid_hash_similarity_coordinates,
)

# Rome coordinates
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88
ROME_META_FILE = "../dataset/rome/output/META-50.txt"

# Initialize a Folium map at a central point
map = folium.Map(location=[41.9, 12.5], zoom_start=12)

# Mark boundaries of Rome
folium.Marker([R_MAX_LAT, R_MAX_LON]).add_to(map)
folium.Marker([R_MIN_LAT, R_MIN_LON]).add_to(map)


def read_coordinates(file_path):
    coordinates = []
    with open("../dataset/rome/output/" + file_path, "r") as file:
        count = 0
        for line in file:
            count += 1
            lat, lon = line.strip().split(", ")
            coordinates.append((float(lat), float(lon)))
        # print(f"Number of coordinates for {file}: {str(count)}")
    return coordinates


# Read the index file to get the list of files containing coordinates
with open(ROME_META_FILE, "r") as index_file:
    number_of_trajectories = 0
    for file_name in index_file:
        if number_of_trajectories == 0:
            break
        file_name = file_name.strip()  # Remove newline characters

        coordinates = read_coordinates(file_name)
        # Plot each coordinate on the map
        for lat, lon in coordinates:
            folium.CircleMarker(
                [lat, lon], radius=1, color="black", fill=False, fill_color="red"
            ).add_to(map)
        folium.PolyLine(coordinates, color="black", weight=1, opacity=1).add_to(map)
        number_of_trajectories += 1

hashed_coordinates, grid = generate_grid_hash_similarity_coordinates(
    city="rome", res=0.6, layers=2, measure="dtw", size=50
)

# TODO: use constant from constants when merged
colors = [
    "red",
    "blue",
    "green",
    "purple",
    "orange",
    "darkred",
    "lightblue",
    "beige",
    "darkgreen",
    "cadetblue",
]

for layer in grid:
    color = colors[layer]
    latitudes = grid[layer][0]
    longitudes = grid[layer][1]
    print(f"Layer {layer} latitudes: {latitudes}")
    print(f"Layer {layer} longitudes: {longitudes}")

    for lat in latitudes:
        for lon in longitudes:
            folium.CircleMarker(
                [lat, lon],
                radius=2,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=1,
            ).add_to(map)
    for lat in latitudes:
        polyline = [(lat, lon) for lon in longitudes]
        folium.PolyLine(polyline, color=color, weight=1).add_to(map)

    # Create polylines for longitude lines (vertical grid lines)
    for lon in longitudes:
        polyline = [(lat, lon) for lat in latitudes]
        folium.PolyLine(polyline, color=color, weight=1).add_to(map)

# print(f"\nNumber of hashed coordinates: {len(hashed_coordinates)}")
# print("Hashed coordinates:", hashed_coordinates)
# for trajectory_key, trajectory_hash in hashed_coordinates.items():
#     print(f"Trajectory hash: {trajectory_key} -> {trajectory_hash}")
#     # print("len hash", len(trajectory_hash))
#     for hashed_trajectory_layer in trajectory_hash:
#         print("len hash", len(hashed_trajectory_layer))
#         print(f"Hash: {hashed_trajectory_layer}" )


#     break

# map

Layer 0 latitudes: [41.88157447923973, 41.887000709637476, 41.892426940035214, 41.89785317043296, 41.903279400830705, 41.90870563122844, 41.91413186162619, 41.91955809202393, 41.92498432242167]
Layer 0 longitudes: [12.442100516186288, 12.449339661989786, 12.456578807793282, 12.463817953596779, 12.471057099400277, 12.478296245203774, 12.48553539100727, 12.492774536810767, 12.500013682614265, 12.50725282841776, 12.514491974221258, 12.521731120024755]
Layer 1 latitudes: [41.88528047631902, 41.89070670671676, 41.8961329371145, 41.90155916751225, 41.90698539790999, 41.91241162830773, 41.917837858705475, 41.92326408910321, 41.92869031950096]
Layer 1 longitudes: [12.447044694969307, 12.454283840772804, 12.4615229865763, 12.468762132379798, 12.476001278183295, 12.483240423986793, 12.490479569790288, 12.497718715593786, 12.504957861397283, 12.512197007200779, 12.519436153004277, 12.526675298807774]

Number of hashed coordinates: 50
Hashed coordinates: {'R_ATG': [[[41.89785317043296, 12.44210051